<a href="https://colab.research.google.com/github/changyuhsin1999/Fake_Medical_News_Detection_Tool/blob/main/SentenceTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "Fake_Medical_News_Detection_Tool" # Enter repo name
git_path = 'https://github.com/changyuhsin1999/Fake_Medical_News_Detection_Tool.git'
!git clone "{git_path}"

Cloning into 'Fake_Medical_News_Detection_Tool'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 177 (delta 9), reused 0 (delta 0), pack-reused 160
Receiving objects: 100% (177/177), 5.03 MiB | 3.75 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [4]:
import os
import numpy as np
import pandas as pd
import string
import time
import urllib.request
import zipfile
import torch

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
raw_df = pd.read_csv('/content/Fake_Medical_News_Detection_Tool/data/raw_df.csv')
labels=raw_df.label
x_train,x_test,y_train,y_test=train_test_split(raw_df['full_text'], labels, test_size=0.2, random_state=7)

In [9]:
senttrans_model = SentenceTransformer('all-MiniLM-L6-v2',device=device)

# Create embeddings for training set text
X_train = x_train.values.tolist()
X_train = [senttrans_model.encode(doc) for doc in X_train]

# Create embeddings for test set text
X_test = x_test.values.tolist()
X_test = [senttrans_model.encode(doc) for doc in X_test]

In [10]:
logreg_model = LogisticRegression(solver='saga')
logreg_model.fit(X_train,y_train)
preds = logreg_model.predict(X_train)
acc = sum(preds==y_train)/len(y_train)
print('Accuracy on the training set is {:.3f}'.format(acc))

Accuracy on the training set is 0.994


In [11]:
preds = logreg_model.predict(X_test)
acc = sum(preds==y_test)/len(y_test)
print('Accuracy on the test set is {:.3f}'.format(acc))

Accuracy on the test set is 0.992
